In [4]:
import pandas as pd



In [5]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt

In [6]:
from ipywidgets import IntProgress


## Sentiment Sample

### Load Sample

In [7]:
data_sample = pd.read_csv('../../raw_data/final_database.csv')


/home/lucab/.pyenv/versions/3.8.6/envs/project_delphi/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (0,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
data_sample

,Unnamed: 0,party,tweet_date,author_id,tweet_id,text,source,retweet_count,reply_count,like_count,profile_creation_date,followers_count,following_count,user_tweet_count,location,sentiment
0,0,CDU,2021-08-26 23:59:59,2.025364e+07,1.431044e+18,@Graf_Fencheltee @Darbales1 Hab mich mal zwisc...,Twitter Web App,0.0,0.0,3.0,2009-02-06 17:27:25,419.0,1035.0,18082.0,Deutschland,-2.0
1,1,CDU,2021-08-26 23:56:43,4.715415e+09,1.431043e+18,@phoenix_de @Inselkind16 @KayGottschalk1 @AfDi...,Twitter for Android,0.0,0.0,0.0,2016-01-05 14:26:34,171.0,31.0,6111.0,NaN,1.0
2,2,CDU,2021-08-26 23:55:40,3.214661e+09,1.431043e+18,@_FriedrichMerz Ich übersetze mal kurz: „lasch...,Twitter for iPhone,0.0,0.0,2.0,2015-04-27 20:47:08,40.0,384.0,3369.0,NaN,-2.0
3,3,CDU,2021-08-26 23:54:03,9.239238e+08,1.431042e+18,#Flüchtlinge #Politik Laschet nennt Ende der R...,dlvr.it,0.0,0.0,0.0,2012-11-03 19:52:01,206.0,88.0,138251.0,NaN,1.0
4,4,CDU,2021-08-26 23:53:42,1.381014e+18,1.431042e+18,@peteraltmaier @cducsubt Und wofür war die Bun...,Twitter Web App,0.0,0.0,0.0,2021-04-10 22:40:34,2.0,13.0,467.0,NaN,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707900,276112,CDU,2021-08-27 00:11:45,8.840942e+17,1.431047e+18,"@SearchSquirrel Wenns so richtig rund läuft, z...",Twitter Web App,0.0,0.0,2.0,2017-07-09 16:57:40,1044.0,791.0,62699.0,NaN,1.0
1707901,276113,CDU,2021-08-27 00:10:44,1.418166e+18,1.431046e+18,@georgrestle @ArminLaschet @MarkusBlume Ansons...,Twitter for iPhone,0.0,0.0,0.0,2021-07-22 11:07:59,57.0,374.0,2270.0,Nowhereland,1.0
1707902,276123,CDU,2021-08-27 00:04:41,1.830903e+07,1.431045e+18,@danielreitzig @ArminLaschet Nachdem die CDU b...,Twitter Web App,0.0,0.0,2.0,2008-12-22 15:42:26,309.0,1505.0,14250.0,Planet Erde,1.0
1707903,276124,CDU,2021-08-27 00:01:20,1.830903e+07,1.431044e+18,@abendblatt Herr Röhse hat jahrelang den ÖPNV...,Twitter Web App,0.0,0.0,0.0,2008-12-22 15:42:26,309.0,1505.0,14250.0,Planet Erde,-2.0


### Clean Sample and put Weight on Tweets

In [9]:
def cleaning_sample(csv_path):
    dict_to_numeric = {"negative": -1, "neutral": 0, "positive": 1}
    df_party_name = pd.read_csv(csv_path)
    df_party_name = df_party_name.drop(columns=['Unnamed: 0',
                                                'Unnamed: 0.1',
                                                "id", 
                                                "conversation_id", 
                                                "user_id", 
                                                "language", 
                                                "hashtags", 
                                                "avg_len_of_tweet",
                                                "replies_count"
                                      ])
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('CEST','')
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('UTC','')
    df_party_name["created_at"] = pd.to_datetime(df_party_name["created_at"],   errors='coerce')
    df_party_name["date"] = pd.to_datetime(df_party_name["date"],  errors='coerce')
    df_party_name = df_party_name.loc[(df_party_name['date'] >= "2021-08-15 00:00:00")]
    df_party_name["sentiment"].replace(dict_to_numeric, inplace=True)
    df_party_name = df_party_name.reset_index(drop=True)
    return df_party_name

In [10]:
df_sample = cleaning_sample('../../raw_data/final_database.csv')
df_sample

/tmp/ipykernel_24815/1912405421.py:1: DtypeWarning: Columns (0,9) have mixed types.Specify dtype option on import or set low_memory=False.
  df_sample = cleaning_sample('../../raw_data/final_database.csv')


KeyError: "['Unnamed: 0.1' 'id' 'conversation_id' 'user_id' 'language' 'hashtags'\n 'avg_len_of_tweet' 'replies_count'] not found in axis"

### Function: Engineer Weighted Sentiment Feature

In [ ]:
def sample_feature_engineering(df_sample):
    '''
    Generates the following features: "Weighted Sentiment", "Share of positive tweets", "Share of negative tweets".
    '''
    # Generate "Weighted Sentiment"
    df_sample["likes_count"] = df_sample["likes_count"]+10
    df_sample["retweets_count"] = df_sample["retweets_count"]+10
    df_sample["weighted_sentiment"] = np.log10(df_sample["likes_count"]) * np.log10(df_sample["retweets_count"]) * df_sample["sentiment"]

    # Generate "Share of positive tweets"
    df_sample["share_of_positive_tweets"] = df_sample["sentiment"]    
    dict_only_positive = {-1: 0, 0: 0, 1: 1}
    df_sample["share_of_positive_tweets"].replace(dict_only_positive, inplace=True)
    
    # Generate "Share of negative tweets"
    df_sample["share_of_negative_tweets"] = df_sample["sentiment"]
    dict_only_negative = {-1: 1, 0: 0, 1: 0}
    df_sample["share_of_negative_tweets"].replace(dict_only_negative, inplace=True)
    
    # Return DF
    return df_sample

In [ ]:
df_sample_features = sample_feature_engineering(df_sample)


In [ ]:
def create_grouped_features(df_sample_features):
    df_sample_features["share_of_positive_tweets2"] = df_sample_features["share_of_positive_tweets"]
    df_sample_features["share_of_negative_tweets2"] = df_sample_features["share_of_negative_tweets"]
    df_sample_features = df_sample_features.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
        "weighted_sentiment": "mean",
        "share_of_positive_tweets": "sum",
        "share_of_positive_tweets2": "count",
        "share_of_negative_tweets": "sum",
        "share_of_negative_tweets2": "count",})
    df_sample_features["share_of_positive_tweets"] = df_sample_features["share_of_positive_tweets"] / df_sample_features["share_of_positive_tweets2"]
    df_sample_features["share_of_negative_tweets"] = df_sample_features["share_of_negative_tweets"] / df_sample_features["share_of_negative_tweets2"]
    df_sample_features = df_sample_features.drop(columns=["share_of_positive_tweets2", "share_of_negative_tweets2"])
    return df_sample_features

In [ ]:
df_grouped_sample_features = create_grouped_features(df_sample_features)

In [ ]:
df_grouped_sample_features

## Poll Sample

In [ ]:
data_poll = pd.read_csv('pools_data_2021_v2.csv')
data_poll

## Pandas Engineer Features Table

### Cleaning Function

In [ ]:
def cleaning(party, csv_path):
    df_party_name = pd.read_csv(csv_path)
    df_party_name = df_party_name.drop(columns=[#'Unnamed: 0',
                              "timezone",
                              "username", 
                              "name", 
                              "place", 
                              "urls", 
                              "photos", 
                              "cashtags", 
                              "retweet", 
                              "quote_url",
                              "video",
                              "thumbnail", 
                              "near", 
                              "source", 
                              "user_rt_id", 
                              "user_rt", 
                              "retweet_id", 
                              "reply_to",  
                              "retweet_date", 
                              "translate", 
                              "trans_src", 
                              "trans_dest",
                              "link",
                              "geo",
                              "time",
                              "mentions"
                                 ])
    df_party_name = df_party_name[(df_party_name.created_at.str.len() == 23) | (df_party_name.created_at.str.len() == 24)]
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('CEST','')
    df_party_name['created_at'] = df_party_name['created_at'].str.replace('UTC','')
    df_party_name["created_at"] = pd.to_datetime(df_party_name["created_at"])
    df_party_name["date"] = pd.to_datetime(df_party_name["date"])
    df_party_name = df_party_name.loc[(df_party_name['date'] >= "2021-08-15 00:00:00")]
    df_party_name = df_party_name[(df_party_name.language == "de")]
    df_party_name = df_party_name.drop_duplicates()
    df_party_name.insert(loc=0, column="party", value=party)
    df_party_name = df_party_name.reset_index(drop=True)

    return df_party_name

In [ ]:
df_cdu = cleaning("CDU",'../raw_data/drive-download-20210826T100733Z-001/CDU_2308_v1.csv')

In [ ]:
df_afd = cleaning("AFD",'../raw_data/drive-download-20210826T100733Z-001/afd_real.csv')

In [ ]:
df_fdp = cleaning("FDP", '../raw_data/drive-download-20210826T100733Z-001/fdp.csv')

In [ ]:
df_gruene = cleaning("GRUENE", '../raw_data/drive-download-20210826T100733Z-001/gruene.csv')

In [ ]:
df_linke = cleaning("LINKE", '../raw_data/drive-download-20210826T100733Z-001/linke.csv')

In [ ]:
df_spd = cleaning("SPD", '../raw_data/drive-download-20210826T100733Z-001/spd.csv')

### Concat Function

In [19]:
liste = [df_afd, df_cdu, df_fdp, df_gruene, df_linke, df_spd]

In [20]:
def concat(liste):
    df_all = pd.concat(liste)
    df_all = df_all.reset_index(drop=True)
    df_all["avg_len_of_tweet"] = df_all["tweet"].str.len()
    return df_all

In [21]:
df_all = concat(liste)

### Feature Engineering

In [22]:
df_all

,party,id,conversation_id,created_at,date,user_id,tweet,language,replies_count,retweets_count,likes_count,hashtags,Unnamed: 0,avg_len_of_tweet
0,AFD,1429944076061839367,1429944076061839367,2021-08-24 01:10:09,2021-08-24,1425768268866936835,Das war mein Kanzler! Und jetzt wähle ich AfD....,de,0.0,0.0,0.0,['afd'],NaN,160
1,AFD,1429943745571696642,1429538422125961221,2021-08-24 01:08:50,2021-08-24,1130242490492182528,@Phnixauschemni1 @Rosenro88130026 @AfD @Alice_...,de,0.0,0.0,0.0,[],NaN,283
2,AFD,1429943687635689473,1427927572977958919,2021-08-24 01:08:36,2021-08-24,241166049,@MarkPrecht @AfD Du wählst die AfD also nicht?,de,0.0,0.0,0.0,[],NaN,46
3,AFD,1429943518034862080,1427927572977958919,2021-08-24 01:07:56,2021-08-24,241166049,"@codeecke @AfD Du weißt anscheinend nicht, wie...",de,0.0,0.0,0.0,[],NaN,89
4,AFD,1429943276895936520,1429943276895936520,2021-08-24 01:06:58,2021-08-24,241166049,"Sehr schönes Video! Wenn #Grüne mitregieren, ...",de,0.0,0.0,0.0,"['grüne', 'lockdown', 'afd', 'deutschlandabern...",NaN,274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322688,SPD,1426953005061091331,1426949316401410056,2021-08-15 19:04:42,2021-08-15,46327787,"@OlafScholz Sie ist so dramatisch, weil Ihr so...",de,1.0,0.0,26.0,[],NaN,81
322689,SPD,1426952979190530049,1426948683170467845,2021-08-15 19:04:36,2021-08-15,1325444836611289090,@stran_d @BILD @spdde @spdbt @Die_Gruenen Das ...,de,1.0,0.0,0.0,[],NaN,196
322690,SPD,1426952826740162562,1425852582073876484,2021-08-15 19:04:00,2021-08-15,1176581940838903808,@Ver_Nunft @NativeYodel @theoneandbodi @FontiF...,de,1.0,0.0,0.0,[],NaN,359
322691,SPD,1426952744515080201,1426518324482134018,2021-08-15 19:03:40,2021-08-15,312038412,@PhilRobinWeber @spdde @Die_Gruenen Das perfid...,de,0.0,0.0,1.0,[],NaN,186


In [23]:
def pandas_feature_creation(df):
    
    # Create: "Avg. len per tweet of each party"
    df_temp = df.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "replies_count": "sum", 
    "retweets_count": "sum",
    "likes_count": "sum",
    "avg_len_of_tweet": "mean"
    }) 
    
    # Create: share of tweets that a party has in comparison to all tweets on a given day 
    df_temp_2 = df_all.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "tweet": "count"}).groupby(level=0).apply(lambda x: x/x.sum())
    
    # Create: Share of tweets that come from a unique user for each party on a given day
    df_temp_3 = df_all.groupby([pd.Grouper(key='created_at',freq='D'), "party"]).agg({
    "user_id": "nunique",
    "tweet": "count"})
    df_temp_3["share_unique_users"] = df_temp_3["user_id"] / df_temp_3["tweet"]
    df_temp_3 = df_temp_3["share_unique_users"]
    
     # Join the different temporary DFs into a final DataFrame
    df_final = df_temp.join(df_temp_2).join(df_temp_3)
    df_final = df_final.rename(columns={'tweet': "share_of_tweets"})
    
    return df_final
    

### Table with pandas features

In [39]:
data_features = pandas_feature_creation(df_all)


KeyError: 'The grouper name created_at is not found'

In [40]:
data_features.shape

(59, 6)

In [25]:
df_all = data_features.join(df_grouped_sample_features)

## RNN Model Table

In [26]:
def create_rnn_final_table(data_poll,df_all):
    data_poll = data_poll.rename(columns = {"Date": "created_at", "Linke": "LINKE", "AfD" : "AFD", "Grüne" : "GRUENE", "CDU/CSU":"CDU"})
    data_poll["created_at"] = pd.to_datetime(data_poll["created_at"])
    data_poll = data_poll.set_index("created_at")
    
    data_poll = pd.DataFrame(data_poll.T.unstack(level = 0))
    data_poll.index = data_poll.index.set_names(['created_at', 'party'])
    
    df_final = data_poll.join(df_all, how = "outer")
    
    df_final = df_final.rename(columns = {0: "Polls"})
    
    return df_final
    

In [27]:
df_final = create_rnn_final_table(data_poll,df_all)

In [28]:
# data_poll = data_poll.rename(columns = {"Date": "created_at", "Linke": "LINKE", "AfD" : "AFD", "Grüne" : "GRUENE", "CDU/CSU":"CDU"})
# data_poll["created_at"] = pd.to_datetime(data_poll["created_at"])
# data_poll = data_poll.set_index("created_at")

In [29]:
# df_pool = pd.DataFrame(data_poll.T.unstack(level = 0))

In [30]:
# df_pool.index = df_pool.index.set_names(['created_at', 'party'])

In [31]:
# df_final = df_pool.join(df_all, how = "outer")

In [32]:
# df_final = df_final.rename(columns = {0: "Polls"})

In [35]:
df_final

Polls  replies_count  retweets_count  \
created_at party                                                     
2021-04-26 AFD                  9.0            NaN             NaN   
           CDU                 24.0            NaN             NaN   
           FDP                 11.0            NaN             NaN   
           GRUENE              29.0            NaN             NaN   
           LINKE                6.0            NaN             NaN   
...                             ...            ...             ...   
2021-08-24 GRUENE              18.0           15.0             5.0   
           LINKE                6.0            5.0             1.0   
           SPD                 23.0           31.0            18.0   
           Surveyed_Persons  2504.0            NaN             NaN   
           other                9.0            NaN             NaN   

                             likes_count  avg_len_of_tweet  share_of_tweets  \
created_at party                                                              
2021-04-26 AFD                       NaN               NaN              NaN   
           CDU                       NaN               NaN              NaN   
           FDP                       NaN               NaN              NaN   
           GRUENE                    NaN               NaN              NaN   
           LINKE                     NaN               NaN              NaN   
...                                  ...               ...              ...   
2021-08-24 GRUENE                   23.0        195.392857         0.245614   
           LINKE                     5.0        150.962963         0.078947   
           SPD                      73.0        201.070866         0.371345   
           Surveyed_Persons          NaN               NaN              NaN   
           other                     NaN               NaN              NaN   

                             share_unique_users  weighted_sentiment  \
created_at party                                                      
2021-04-26 AFD                              NaN                 NaN   
           CDU                              NaN                 NaN   
           FDP                              NaN                 NaN   
           GRUENE                           NaN                 NaN   
           LINKE                            NaN                 NaN   
...                                         ...                 ...   
2021-08-24 GRUENE                      0.821429           -0.279951   
           LINKE                       0.777778           -0.296296   
           SPD                         0.740157           -0.426771   
           Surveyed_Persons                 NaN                 NaN   
           other                            NaN                 NaN   

                             share_of_positive_tweets  \
created_at party                                        
2021-04-26 AFD                                    NaN   
           CDU                                    NaN   
           FDP                                    NaN   
           GRUENE                                 NaN   
           LINKE                                  NaN   
...                                               ...   
2021-08-24 GRUENE                            0.095238   
           LINKE                             0.074074   
           SPD                               0.023622   
           Surveyed_Persons                       NaN   
           other                                  NaN   

                             share_of_negative_tweets  
created_at party                                       
2021-04-26 AFD                                    NaN  
           CDU                                    NaN  
           FDP                                    NaN  
           GRUENE                                 NaN  
           LINKE                                  NaN  
...                              

In [34]:
df_final.to_csv("df_mergedv2.csv")